--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

(Example see Input see next Cell)

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


In [107]:
input_str = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [108]:
def read_input(input_str):
    lines = input_str.split('\n')
    lines = [line for line in lines if line.strip()]
    return lines

def init_2d_array(lines):
    array_2d = []
    for line  in lines:
        row = []
        for char in line:
            row.append(char)
        array_2d.append(row)
    return array_2d

def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]
    return lines

In [109]:
## Cases for Adjacency
# Left (x-1, y)
# Right (x+1, y)
# Up (x, y-1)
# Down (x, y+1)
# Up-Left (x-1, y-1)
# Up-Right (x+1, y-1)
# Down-Left (x-1, y+1)
# Down-Right (x+1, y+1)



def has_adjacent_symbol(arr_2d, x, y):
    rows = len(arr_2d)
    cols = len(arr_2d[0]) if rows > 0 else 0

    if x > 0 and not (arr_2d[x-1][y].isdigit() or arr_2d[x-1][y] == "."):
        return True
    if x < rows - 1 and not (arr_2d[x+1][y].isdigit() or arr_2d[x+1][y] == "."):
        return True
    if y > 0 and not (arr_2d[x][y-1].isdigit() or arr_2d[x][y-1] == "."):
        return True
    if y < cols - 1 and not (arr_2d[x][y+1].isdigit() or arr_2d[x][y+1] == "."):
        return True
    if x > 0 and y > 0 and not (arr_2d[x-1][y-1].isdigit() or arr_2d[x-1][y-1] == "."):
        return True
    if x < rows - 1 and y > 0 and not (arr_2d[x+1][y-1].isdigit() or arr_2d[x+1][y-1] == "."):
        return True
    if x > 0 and y < cols - 1 and not (arr_2d[x-1][y+1].isdigit() or arr_2d[x-1][y+1] == "."):
        return True
    if x < rows - 1 and y < cols - 1 and not (arr_2d[x+1][y+1].isdigit() or arr_2d[x+1][y+1] == "."):
        return True

    return False

def get_part_numbers(arr_2d):
    part_numbers = []
    for i in range(len(arr_2d)):
        currnum = ""
        for j in range(len(arr_2d[i])):
            if arr_2d[i][j].isdigit():
                currnum += arr_2d[i][j]
                # Special check for EOL Numbers (didnt work before since i was checking after the number ended)
                if j == len(arr_2d[i]) - 1 and currnum:
                    if has_adjacent_symbol(arr_2d, i, j - len(currnum) + 1):
                        part_numbers.append(int(currnum))
                    currnum = ""
            else:
                if currnum:
                    # Check for symbol adjacent to any digit of the number
                    for k in range(len(currnum)):
                        if has_adjacent_symbol(arr_2d, i, j - k - 1):
                            part_numbers.append(int(currnum))
                            break
                    currnum = ""

    return part_numbers
                    

In [113]:
input_file_path = 'input.txt'
lines = read_input_from_file(input_file_path)
array_2d = init_2d_array(lines)
sum(get_part_numbers(array_2d))

527364

In [114]:
def calculate_gear_ratios_fixed(arr_2d):
    gear_ratios = []
    
    def extract_full_number(x, y):
        number = ""
        original_y = y
        while y > 0 and arr_2d[x][y-1].isdigit():
            y -= 1
        # Extract the full number
        while y < len(arr_2d[0]) and arr_2d[x][y].isdigit():
            number += arr_2d[x][y]
            y += 1
        # To prevent double counting, mark the entire number as non-digit if it was used
        if y != original_y:
            for mark_y in range(y - len(number), y):
                arr_2d[x][mark_y] = 'X'  # Replacing digits with 'X' to avoid re-counting
        return int(number) if number else 0

    for i in range(len(arr_2d)):
        for j in range(len(arr_2d[i])):
            if arr_2d[i][j] == '*':
                # Find all unique part numbers adjacent to the gear
                adjacent_numbers = []
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        if dx == 0 and dy == 0:
                            continue
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < len(arr_2d) and 0 <= nj < len(arr_2d[0]) and arr_2d[ni][nj].isdigit():
                            number = extract_full_number(ni, nj)
                            if number not in adjacent_numbers:
                                adjacent_numbers.append(number)

                # Calculate the gear ratio if there are exactly two adjacent part numbers
                if len(adjacent_numbers) == 2:
                    gear_ratio = adjacent_numbers[0] * adjacent_numbers[1]
                    gear_ratios.append(gear_ratio)

    return sum(gear_ratios)

sum_gear_ratios_fixed = calculate_gear_ratios_fixed(array_2d)
sum_gear_ratios_fixed

79026871